In [1]:
library(Matching)
library(tidyverse)

Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
201220,125078,66690,0,2106-04-27 09:47:50,2106-05-01 19:25:46,1,62.67646,1,NA,⋯,1,7.45,7.42,7.45,1,1,0,1,0.4544669,1.833069
215842,151232,11663,0,2188-02-14 09:48:15,2188-02-16 03:02:48,1,86.76186,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.1568838,1.186076
234312,164444,86645,1,2165-06-22 09:47:16,2165-07-07 21:55:20,1,56.08904,1,NA,⋯,1,7.49,7.49,7.49,1,0,0,0,0.3002571,1.429096
289157,146726,10304,0,2156-06-23 22:26:00,2156-06-30 17:26:00,1,45.91093,1,2156-06-25 08:00:00,⋯,1,7.44,7.44,7.44,1,0,1,0,0.6991412,1.430326
211964,160170,94534,0,2160-03-05 22:23:19,2160-03-07 06:48:41,1,59.38693,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.3432285,1.522600
230173,112553,31544,0,2140-01-31 04:39:25,2140-02-03 02:41:39,1,91.50000,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2464983,1.327137


In [ ]:
set.seed(4958)

In [4]:
ps_matches <- Match(Y = full_data$mort_28_day_int, Tr = full_data$echo_int,
                    X = full_data$ps, M = 1, estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE, ties = FALSE)

In [ ]:
summary(ps_matches)

In [5]:
outcome <- data.frame(echo_pt=full_data$mort_28_day_int[ps_matches$index.treated],
                      match_pt=full_data$mort_28_day_int[ps_matches$index.control])

In [6]:
head(outcome)

echo_pt,match_pt
0,0
1,0
1,0
1,1
0,1
1,1


In [8]:
tab <- table(outcome$echo_pt, outcome$match_pt, dnn = c("Echo","Non-Echo"))
tab
tab[2, 1] / tab[1, 2]
paste("95% Confint",
      round(exp(c(log(tab[2, 1] / tab[1, 2]) - qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]),
                  log(tab[2, 1] / tab[1, 2]) + qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]))), 7))
mcnemar.test(tab) # for 1-1 pairs

811,351
281,105


[1] 0.8005698

[1] "95% Confint 0.6843244" "95% Confint 0.9365617"


	McNemar's Chi-squared test with continuity correction

data:  tab
McNemar's chi-squared = 7.5332, df = 1, p-value = 0.006057


In [9]:
(tab[2, 2] + tab[1, 2]) / sum(tab)
(tab[2, 2] + tab[2, 1]) / sum(tab)

[1] 0.2945736

[1] 0.249354

In [10]:
mortality <- data.frame(echo = c(0, 1),
                        mortality = c((tab[2, 2] + tab[1, 2]) / sum(tab),
                                      (tab[2, 2] + tab[2, 1]) / sum(tab)))
mortality

echo,mortality
0,0.2945736
1,0.2493540


In [11]:
mort_fmt <- mortality %>%
mutate(echo = case_when(
    echo == 0 ~ "Non_echo",
    echo == 1 ~ "Echo"
)) %>%
select(echo, mortality) %>%
spread(echo, mortality) %>%
mutate(Method = "PS Match") %>%
select(Method, Echo, Non_echo)
mort_fmt

Method,Echo,Non_echo
PS Match,0.249354,0.2945736


In [12]:
data.table::fwrite(mort_fmt, file.path(data_dir, "mort_ps.csv"))